# Import

## Libraries

In [1]:
# Warning control
import warnings
import os
import yaml
from pathlib import Path


from dotenv import load_dotenv
from crewai import Agent, Task, Crew
from crewai_tools import FileReadTool, DirectoryReadTool

from pydantic import BaseModel, Field
from typing import List, Optional
from IPython.display import display, Markdown


load_dotenv()  # take environment variables from .env
warnings.filterwarnings("ignore")

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"

## Conversation Transcript

Output from `pdf_to_md.ipynb`:

In [2]:
transcript_content = Path("data/conversation_01.txt").read_text()
print(transcript_content[:500])

CSR: Good morning! Thank you for calling ShiokAh Insurance. My name is Sarah. How may I assist you today?

Customer: Hi Sarah! I want to check about travel insurance leh. Planning to go Japan next month.

CSR: I'll be happy to help you with that. Before we look into the options, could you share some information with me? May I know your age?

Customer: I'm 62 this year.

CSR: Thank you for sharing. Since you're above 60, we'll need to note this for coverage considerations. Do you have any pre-exi


## Tasks & Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    "agents": "src/recsys/config/agents.yaml",
    "tasks": "src/recsys/config/tasks.yaml",
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, "r") as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs["agents"]
tasks_config = configs["tasks"]

In [4]:
print(
    "agents_config contains:",
    ", ".join(agents_config.keys()),
    "\ntasks_config contains:",
    ", ".join(tasks_config.keys()),
)

agents_config contains: conversation_transcript_agent, underwriting_report_agent 
tasks_config contains: process_transcript_task, generate_underwriting_report_task


## Tools (Agent Usage)

Got `FileReadTool()` to work, thanks to this [article](https://lablab.ai/t/crewai-multi-agent-system)!

In [ ]:
directory_read_tool = DirectoryReadTool(directory="./data")
file_read_tool = FileReadTool()


# read_conversation = FileReadTool(file_path="./data/conversation_01.txt")
# read_income = FileReadTool(file_path="./data/income_travel_insurance_2024.txt")
# read_ge = FileReadTool(file_path="data/ge_travel_insurance_2024.txt")

# Pydantic Classes

In [6]:
class CustomerInfo(BaseModel):
    age: int
    medical_conditions: Optional[List[str]] = Field(default_factory=list)
    destination: str
    coverage_factors: List[str] = Field(default_factory=list)
    policy_customization_factors: List[str] = Field(default_factory=list)


class CustomerInfoList(BaseModel):
    customers: List[CustomerInfo] = Field(default_factory=list)

# Create Crew, Agents and Tasks

In [7]:
conversation_transcript_agent = Agent(
    config=agents_config["conversation_transcript_agent"],
    tools=[directory_read_tool, file_read_tool],
    verbose=True,
)

underwriting_report_agent = Agent(
    config=agents_config["underwriting_report_agent"],
    tools=[directory_read_tool, file_read_tool],
    verbose=True,
)

process_transcript_task = Task(
    config=tasks_config["process_transcript_task"],
    agent=conversation_transcript_agent,
    output_pydantic=CustomerInfoList,
)

generate_underwriting_report_task = Task(
    config=tasks_config["generate_underwriting_report_task"],
    context=[process_transcript_task],
    agent=underwriting_report_agent,
    async_execution=True,
)

In [8]:
recsys_crew = Crew(
    agents=[conversation_transcript_agent, underwriting_report_agent],
    tasks=[
        process_transcript_task,
        generate_underwriting_report_task,
    ],
    verbose=True,
)

In [9]:
result = recsys_crew.kickoff()
display(Markdown(result.raw))


# Agent: Conversation Transcript Analyst (Travel Insurance)
## Task: This task involves analyzing a conversation transcript between a potential customer interested in travel insurance and a customer service representative from ShiokAh Insurance. The goal is to extract key customer details from the transcript, such as age, medical conditions, destination, coverage preferences, and customization requests. This extracted information will be organized into a Python list and used by an underwriter to determine policy suitability.


# Agent: Conversation Transcript Analyst (Travel Insurance)
## Thought: To accomplish the task of extracting customer details from the conversation transcript, I need to first access and read the content of the file {{conversation_01.txt}}.
## Using tool: List files in directory
## Tool Input: 
"{}"
## Tool Output: 
File paths: 
-./data/income_travel_insurance_2024.txt
- ./data/conversation_01.txt
- ./data/ge_travel_insurance_2024.txt


# Agent: Conversation Tran

```markdown
# Policy Summary

Income's Travel Insurance provides comprehensive travel coverage with various plans (Classic, Deluxe, Preferred). Key policy highlights include coverage for trip cancellation, medical expenses, emergency medical evacuation, travel delay, and adventurous activities. Notably, standard plans do not cover pre-existing medical conditions, but the Enhanced PreX Travel Insurance offers this option. The policy covers leisure activities like hiking and scuba diving, subject to safety guidelines and certain conditions.

## Age

- **Customer Age**: 62
- **Policy Implication**: The policy specifies different coverage limits for individuals aged 70 or older and those under 70. At 62, the customer is considered under 70, allowing access to higher coverage limits in several sections, including personal accident and medical expenses benefits.

## Medical Conditions

- **Customer Medical Conditions**: High blood pressure, diabetes (controlled with medication).
- **Policy Consideration**: Standard plans do not cover pre-existing medical conditions. Coverage options for medical emergencies abroad, up to $1,000,000 under the Preferred plan, are valuable. If medical conditions trigger costs, the customer needs to consider the Enhanced PreX Travel Insurance for additional coverage specific to pre-existing conditions.

## Destination

- **Destination**: Japan
- **Duration**: 2 weeks (15th December - 29th December)
- **Policy Implication**: Coverage for travel delays, medical expenses, and emergency evacuation will be crucial due to the international nature of travel. Japan's sophisticated healthcare system aligns well with the policy's comprehensive medical expense coverage.

## Coverage Factors

- **Requested Coverages**:
  - Emergency Medical Evacuation: Unlimited for those under 70.
  - Hospital Daily Cash Benefit: Up to $150,000 limit.
  - Outpatient Treatment: Included.
  - Flight Delay: Coverage starts from delays exceeding six hours.
  - Luggage Loss: Up to $8,000 under Preferred plan.
  - Trip Cancellation: Cancelling benefits up to $15,000 under Preferred plan.
- **Policy Offering**: The requested coverages align well with the Preferred plan, offering extensive protection for overseas medical and non-medical incidents. The automatic extension for unforeseen delays adds valuable flexibility.

## Policy Customization Factors

- **Activities**: Light hiking, onsen visit, sightseeing.
- **Companions**: Wife (age 60, no medical conditions)
- **Excess Preference**: Lower excess
- **Policy Implication**: Hiking is covered under the adventurous activities option provided it follows safety guidelines. The couple is traveling with a spouse without medical conditions, making joint policy applications likely eligible for spouse or family plans, enabling potential savings on premiums. Lower excess preference indicates a trade-off for higher premiums in exchange for reduced out-of-pocket expenses in case of claims.

This structured report cross-references the customer's details with Income's Travel Insurance policy to highlight coverage, limits, and policy choices relevant for creating an efficient travel insurance plan.
```